In [ ]:
! rm -rf sample_data; curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/sync_hw.py > sync_hw.py
files = ["find_throwing_trajectory.py", "dynamics_constraints.py", "kinematic_constraints.py", "planar_arm.urdf"]

from sync_hw import sync_hw
sync_hw(5, files, False)

# Problem 4 - Trajectory Optimization for a planar arm

In [ ]:
# Install drake. We are using the lqr controller in drake as the final stabilizing controller.
# The installation process will take about 2 minutes but it's only required in the start of the Colab's virtual machine.
!curl -s https://raw.githubusercontent.com/mposa/MEAM517/master/colab_drake_setup.py > colab_drake_setup.py
from colab_drake_setup import setup
setup()


# Install pyngrok.
server_args = []
!pip install pyngrok
server_args = ['--ngrok_http_tunnel']

# TODO(russt): Consider teaching meshcat how to open the window on colab.
def open_window(url, name):
    # This has the benefit of not persisting between notebooks if the output 
    # is accidentally saved.
    from google.colab import output
    output.eval_js(f'window.open("{url}", name);', ignore_result=True)

In [ ]:
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Compute the end effector (the ball) position and velocity using symbolic variables

This is how we get the expression in `kinematic_constraints.py`

In [ ]:
# Computes the end effector velocity as xdot, ydot J_q (pos) * qdot

from sympy import *
import numpy as np

l, q0, q1, q2, qdot0, qdot1, qdot2 = symbols('l q0 q1 q2 qdot0 qdot1 qdot2', real=True)
pos = l * np.array([sin(q0), -cos(q0)]) + l * np.array([sin(q0 + q1), -cos(q0 + q1)]) + l * np.array([sin(q0 + q1 + q2), -cos(q0 + q1 + q2)])
vel = (Matrix(pos).jacobian([q0, q1, q2]) @ np.reshape(np.array([qdot0, qdot1, qdot2]), (3,1)))

print('pos (x_ball, z_ball):', pos)
print('vel (x_ball dot, z_ball dot):', vel)

# Run trajectory optimization to find the optimal trajectory

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import importlib

from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator

from pydrake.all import (
    DiagramBuilder, Simulator
)

from pydrake.multibody.tree import (
    JointActuatorIndex
)

from pydrake.geometry import SceneGraph
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.trajectories import PiecewisePolynomial
from pydrake.solvers.snopt import SnoptSolver


import kinematic_constraints
import dynamics_constraints
importlib.reload(kinematic_constraints)
importlib.reload(dynamics_constraints)
from kinematic_constraints import (
  AddFinalLandingPositionConstraint
)
from dynamics_constraints import (
  AddCollocationConstraints,
  EvaluateDynamics
)


def find_throwing_trajectory(N, initial_state, distance, tf):
  '''
  Parameters:
    N - number of knot points
    initial_state - starting configuration
    distance - target distance to throw the ball

  '''

  builder = DiagramBuilder()
  plant = builder.AddSystem(MultibodyPlant(0.0))
  file_name = "planar_arm.urdf"
  Parser(plant=plant).AddModelFromFile(file_name)
  plant.Finalize()
  planar_arm = plant.ToAutoDiffXd()

  plant_context = plant.CreateDefaultContext()
  context = planar_arm.CreateDefaultContext()

  # Dimensions specific to the planar_arm
  n_x = planar_arm.num_positions() + planar_arm.num_velocities()
  n_u = planar_arm.num_actuators()

  # Store the actuator limits here
  effort_limits = np.zeros(n_u)
  for act_idx in range(n_u):
    effort_limits[act_idx] = \
      planar_arm.get_joint_actuator(JointActuatorIndex(act_idx)).effort_limit()
  vel_limits = 15 * np.ones(n_x // 2)

  # Create the mathematical program
  prog = MathematicalProgram()
  x = np.zeros((N, n_x), dtype="object")
  u = np.zeros((N, n_u), dtype="object")
  for i in range(N):
    x[i] = prog.NewContinuousVariables(n_x, "x_" + str(i))
    u[i] = prog.NewContinuousVariables(n_u, "u_" + str(i))

  t_land = prog.NewContinuousVariables(1, "t_land")

  t0 = 0.0
  timesteps = np.linspace(t0, tf, N)
  x0 = x[0]
  xf = x[-1]


  # Add the kinematic constraints (initial state, final state)
  # TODO: Add constraints on the initial state
  
  # Add the kinematic constraint on the final state
  AddFinalLandingPositionConstraint(prog, xf, distance, t_land)

  # Add the collocation aka dynamics constraints
  AddCollocationConstraints(prog, planar_arm, context, N, x, u, timesteps)

  # TODO: Add the cost function here

  # TODO: Add bounding box constraints on the inputs and qdot 

  # TODO: give the solver an initial guess for x and u using prog.SetInitialGuess(var, value)

  # Set up solver
  solver = SnoptSolver()
  result = solver.Solve(prog)
  
  x_sol = result.GetSolution(x)
  u_sol = result.GetSolution(u)
  t_land_sol = result.GetSolution(t_land)

  print(result.get_solution_result())

  # Reconstruct the trajecotry as a cubic hermite spline
  xdot_sol = np.zeros(x_sol.shape)
  for i in range(N):
    xdot_sol[i] = EvaluateDynamics(plant, plant_context, x_sol[i], u_sol[i])
  
  x_traj = PiecewisePolynomial.CubicHermite(timesteps, x_sol.T, xdot_sol.T)
  u_traj = PiecewisePolynomial.ZeroOrderHold(timesteps, u_sol.T)

  return x_traj, u_traj, prog, prog.GetInitialGuess(x), prog.GetInitialGuess(u)
  
if __name__ == '__main__':
  N = 7
  initial_state = np.zeros(6)
  tf = 5.0
  distance = 25.0
  x_traj, u_traj, prog, x_guess, u_guess = find_throwing_trajectory(N, initial_state, distance, tf)


# Visualize the optimal trajectory

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator
from pydrake.systems.drawing import plot_system_graphviz

from pydrake.all import ( 
  ConnectMeshcatVisualizer, DiagramBuilder, 
  Simulator
)

from pydrake.geometry import (
  SceneGraph, ConnectDrakeVisualizer
)

from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.systems.rendering import MultibodyPositionToGeometryPose
from pydrake.systems.primitives import (
  TrajectorySource,
  Demultiplexer,
  ConstantVectorSource
)

# """Cart-Pole with simple geometry."""
file_name = "planar_arm.urdf"
builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
planar_arm = builder.AddSystem(MultibodyPlant(0.0))
planar_arm.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant=planar_arm).AddModelFromFile(file_name)
planar_arm.Finalize()

# Create meshcat
visualizer = ConnectMeshcatVisualizer(
    builder, 
    scene_graph, 
    scene_graph.get_pose_bundle_output_port(),
    zmq_url="new",
    server_args=server_args)

x_traj_source = builder.AddSystem(TrajectorySource(x_traj))
u_traj_source = builder.AddSystem(TrajectorySource(u_traj))

demux = builder.AddSystem(Demultiplexer(np.array([3, 3])))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(planar_arm))
zero_inputs = builder.AddSystem(ConstantVectorSource(np.zeros(3)))

builder.Connect(zero_inputs.get_output_port(), planar_arm.get_actuation_input_port())
builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(planar_arm.get_source_id()))

ConnectDrakeVisualizer(builder, scene_graph);

diagram = builder.Build()
diagram.set_name("diagram")

# Set up a simulator to run this diagram.

# Open the meshcat visualizer window (check your pop-up blocker).
open_window(visualizer.vis.url(), "meshcat")

for i in range(3):
  simulator = Simulator(diagram)
  simulator.Initialize()
  simulator.set_target_realtime_rate(0.5)
  simulator.AdvanceTo(tf);
  time.sleep(5)
